In [10]:
import requests 
#import warnings 
import os
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup 
from tqdm import tqdm

#warnings.filterwarnings("ignore")

In [11]:
#gets the urls of the sub-categories
def get_links():
    url = "https://techtop.co.il"
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, 'lxml')
    links=[]
    product_categories = soup.select("div.category-list-item")
    for product_category in product_categories: 
        category_name = product_category.find("p",{"class":"footer-title"}).text.strip()
        ul = product_category.find("ul",{"class":"category-list-hidden"})
        for li in ul.find_all("li"):
            try:
                a_tag = li.find("a")
                sub_category_link = a_tag["href"]
                links.append(sub_category_link)
            except:
                   pass 
    return links







In [12]:
#gets the data-information of all the product pages without a sub-category
def get_catIds(urls):
    test_links = urls
    cat_ids=[]
    list_cat=[]
    for link in test_links:
        url = link
        response=requests.get(url,verify=False)
        doc1=BeautifulSoup(response.text,'html.parser')
        o_tag=doc1.find('h1',class_='woocommerce-products-header__title page-title').text.strip()
        main_cat=doc1.find('section',class_='breadcrumbs-catalog').find_all('li')
        for tag in main_cat:
            li=tag.text.strip()
            list_cat.append(li)
        element=doc1.find('h1',class_='woocommerce-products-header__title page-title')
        if element:
            data_id=element.get('data-term_id')
            cat_ids.append((data_id,o_tag,list_cat[-1]))
        else:
            pass
    return cat_ids

    



In [13]:
#gets the product info of all the products for each category pages
import requests
from bs4 import BeautifulSoup
import pandas as pd
def get_product_info():
    #data_ids=[#('1336', 'Installation Amplifiers', 'AV Installations'),
 #('1337', 'Wall, wall and ceiling subwoofers', 'AV installations'),
 #('1340', 'Processors and Matrix', 'AV Installations'),
 #('1338', 'Flexible Neck and Bases', 'AV Installations'),
 #('1339', 'Home Speakers', 'AV Installations'),
 #('1353', 'Complementary equipment stands and cables', 'DJ studio and musical instruments'),
 #('939', 'studio headphones', 'studio DJ and musical instruments'),
 #('1276', 'Effects & DI', 'Studio DJ & Instruments'),
 #('1052', 'Musical Instruments', 'DJ Studio and Musical Instruments'),
 #('941', 'Sound Cards', 'DJ Studio and Musical Instruments'),
 #('942', 'studio monitors', 'studio DJ and musical instruments'),
# ('1112', 'Microphones for the studio', 'DJ studio and musical instruments'),
# ('1083', 'Keyboards and controllers', 'DJ studio and musical instruments'),
 #('943', 'Acoustic solutions', 'DJ studio and musical instruments'),
 #('1045', 'Speakers and amplifiers for musical instruments', 'DJ studio and musical instruments'),
 #('1306', 'measuring instruments', 'sound amplification'),
 #('765', 'headset', 'volume up'),
#('743', 'Amplifiers', 'Voice Amplification'),
 #('752', 'Mixers', 'Voice Boost'),
 #('770', 'Microphones', 'Voice Boost'),
 #('760', 'Wireless Microphones', 'Voice Boost'),
 #('1080', 'Translation and Training Accessibility Systems', 'Voice Amplification'),
 #('793', 'players / recorders', 'amplifying sound'),
 #('999', 'Processors, Processors and Control', 'Voice Amplification'),
#('745', 'Speakers', 'Volume Up'),
# ('1035', 'Cable rolls for audio and lighting', 'Cables and connectors'),
# ('786', 'A/V Cables', 'Cables and Connectors'),
 #('787', 'network cables', 'cables and connectors'),
# ('1120', 'cables with connectors', 'cables and connectors'),
# ('1288', 'connectors', 'cables and connectors'),
 #('1298', 'A/V Adapters and Switches', 'Cables and Connectors'),
 #('680', 'cable testers', 'cables and connectors'),
 #('1069', 'Azerim', 'Cables and Connectors'),
 #('1330', 'Projection screens', 'Multimedia/AV'),
 #('1334', 'Interactive screens', 'Multimedia/AV'),
 #('1309', 'Multimedia stands/accessories', 'Multimedia/AV'),
 #('1061', "Gadgets and Computers", 'Multimedia/AV'),
 #('949', 'Matrix', 'Multimedia/AV'),
# ('950', 'Converters', 'Multimedia/AV'),
 #('955', 'Switches and Splitters', 'Multimedia/AV'),
 #('962', 'Cameras', 'Multimedia/AV'),
# ('963', 'Projectors', 'Multimedia/AV'),
# ('956', 'Remote', 'Multimedia/AV'),
# ('957', 'AV panels and boxes', 'Multimedia/AV'),
 #('960', 'Presentation', 'Multimedia/AV'),
 #('1299', 'AV Control', 'Multimedia/AV'),
# ('1031', 'Zivod RACK wardrobe accessories', 'Stands and accessories'),
# ('1030', 'Flight Case', 'Stands and Accessories'),
# ('1300', 'stands and accessories for amplification', 'stands and accessories'),
# ('799', 'Stands and accessories for multimedia', 'Stands and accessories'),
# ('1025', 'Lighting stands and accessories', 'Stands and accessories'),
# ('1269', 'Stands for the stage and musical instruments', 'Stands and accessories'),
 #('1068', 'Power supplies and batteries', 'Stands and accessories'),
# ('1202', 'Cables and connectors for lighting', 'Lighting'),
# ('1220', 'Smoke machine and other effects', 'Lighting'),
 #('1212', 'stands/accessories for hanging lanterns', 'lighting'),
 #('1227', 'Lights and Stage Accessories', 'Lighting'),
 #('1193', 'Lighting commands and controls', 'Lighting'),
 #('1251', 'Effect lights and mirror balls', 'Lighting'),
 #('1226', 'photo lights', 'lighting'),
# ('1021', 'architectural lighting fixtures', 'lighting'),
 #('780', 'motorized lights', 'lighting'),
 #('1242', 'Static lights for stage and theater', 'Lighting')]
    products=[]
    skus=[]
    prices=[]
    product_links=[]
    sub_cats=[]
    main_cats=[]
    for i in range(len(data_ids)):
        url = "https://techtop.co.il/wp-admin/admin-ajax.php"
        payload = {
            "cat":f'{data_ids[i][0]}',
            "pageNumber":"1",
            "ppp":"1",
            "action":"all_post_ajax"
        }
        response = requests.post(url, data=payload, verify=False)
        doc=BeautifulSoup(response.text,'html.parser')
        title=doc.find_all('div',class_='item-info')
        main_category=data_ids[i][2]
        sub_category=data_ids[i][1]
        for tag in title:
            prod=tag.find('p',class_="title woocommerce-loop-product__title").text.strip()
            prod_url=tag.find('a')['href']
            product_links.append(prod_url)
            sku=tag.find('p',class_='sku').text.strip()
            products.append(prod)
            skus.append(sku)
            try:
                price=tag.find('div',class_='item-price').text.strip()
                prices.append(price)
            except:
                price='None'
                prices.append(price)
            sub_cats.append(sub_category)
            main_cats.append(main_category)
        dict={
            'Main_Category':main_category,
            'Sub_Category':sub_category,
            'Product_name':products,
            'sku':skus,
            'prices':prices,
            'product_links':product_links


        } 
        filename=f'Desktop/{main_category}.csv'
        if os.path.exists(filename):
            print(f'This {main_category},{sub_category} file already exists.Skipping........')
        else:
            os.makedirs(filename)
            print(f'Scraping product data for {main_category},{sub_category}')
            with open(filename,'w')as f:
                writer=csv.writer(f)
                writer.writerow(main_catgeory)
                writer.writerow(sub_catgeory)
                writer.writerow(products)
                writer.writerow(skus)
                writer.writerow(prices)
                writer.writerow(product_links)
            print('Done-------------------')

 



     
        
        


In [14]:
#gets the id of the products within a mini-category
def get_Id_miniCat(min_urls):
    test_links = min_urls[:10]
    cat_ids=[]
    list_cat=[]
    for link in test_links:
        url = link
        response=requests.get(url,verify=False)
        doc1=BeautifulSoup(response.text,'html.parser')
        o_tag=doc1.find('h1',class_='woocommerce-products-header__title page-title').text.strip()
        main_cat=doc1.find('section',class_='breadcrumbs-catalog').find_all('li')
        for tag in main_cat:
            li=tag.text.strip()
            list_cat.append(li)
        element=doc1.find('h1',class_='woocommerce-products-header__title page-title')
        if element:
            data_id=element.get('data-term_id')
            cat_ids.append((data_id,o_tag,list_cat[-1],list_cat[2]))
            print(cat_ids)
        else:
            pass

    return cat_ids
        
        
    
    

In [15]:
def get_mini_category(urls):#gets the links of the pages with the mini-categories
    mini_links=[]
    test_links=urls[:10]
    for miin_link in test_links:
        response=requests.get(miin_link,verify=False)
        mini_doc=BeautifulSoup(response.text,'html.parser')
        mini_category=mini_doc.find_all('h3',class_='woocommerce-loop-category__title')
        if mini_category:
            div_tag=mini_doc.find('div',class_='catalog-category catalog-products-list')
            divi_tag=div_tag.find_all('div',class_='category-header')
            for tag in divi_tag:
                a_tag=tag.find('a')['href']
                mini_links.append(a_tag)

        else:
            pass
    return mini_links



In [16]:
#This code will scrape the products for a category with a mini-Category
import requests
from bs4 import BeautifulSoup
import pandas as pd
def get_mini_info(datas_ids):
    products=[]
    skus=[]
    prices=[]
    product_links=[]
    sub_cats=[]
    main_cats=[]
    for i in range(len(datas_ids)):
        url = "https://techtop.co.il/wp-admin/admin-ajax.php"
        payload = {
            "cat":f'{datas_ids[i][0]}',
            "pageNumber":"1",
            "ppp":"1",
            "action":"all_post_ajax"
        }
        response = requests.post(url, data=payload, verify=False)
        doc=BeautifulSoup(response.text,'html.parser')
        title=doc.find_all('div',class_='item-info')
        mini_category=doc.find_all('h3',class_='woocommerce-loop-category__title')
        main_category=datas_ids[i][3]
        sub_category=datas_ids[i][2]
        mini_categories=datas_ids[i][1]
        for tag in title:
            prod=tag.find('p',class_="title woocommerce-loop-product__title").text.strip()
            prod_url=tag.find('a')['href']
            product_links.append(prod_url)
            sku=tag.find('p',class_='sku').text.strip()
            products.append(prod)
            skus.append(sku)
            try:
                price=tag.find('div',class_='item-price').text.strip()
                prices.append(price)
            except:
                price='None'
                prices.append(price)
            sub_cats.append(sub_category)
            main_cats.append(main_category)
            mini_category.append(mini_category)
        
        
        dict={
            'Main_Category':main_category,
            'Sub_Category':sub_category,
            'Mini_Category':mini_categories,
            'Product_name':products,
            'sku':skus,
            'prices':prices,
            'product_links':product_links


        } 
        print(f'Scraping product data for {main_category},{sub_category},{mini_category}')
        filename=f'{main_category}.csv'
        pd.DataFrame(dict).to_csv(filename,index=None)
        print('Done------------------------------------')

   


     
        
        


In [17]:
def scrape_prod():
    df=get_mini_info(get_Id_miniCat(get_mini_category(get_links())))
    return df
    

In [20]:
#with ThreadPoolExecutor(5) as exec:
    #r=exec.map(get_mini_info,get_Id_miniCat(get_mini_category(get_links())))
#r
get_mini_info(get_Id_miniCat(get_mini_category(get_links())))

C:\Users\PETER\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'techtop.co.il'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\PETER\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'techtop.co.il'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\PETER\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'techtop.co.il'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\PETER\anaconda3\lib\site-packages\urllib3\connectionpoo

[('1358', 'אביזרי כלי נגינה ואולפן', 'ציוד משלים מעמדים וכבלים', 'אולפן DJ וכלי נגינה')]


C:\Users\PETER\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'techtop.co.il'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[('1358', 'אביזרי כלי נגינה ואולפן', 'ציוד משלים מעמדים וכבלים', 'אולפן DJ וכלי נגינה'), ('1355', 'כיסויים ותיקים', 'ציוד משלים מעמדים וכבלים', 'אולפן DJ וכלי נגינה')]


C:\Users\PETER\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'techtop.co.il'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

Scraping product data for אולפן DJ וכלי נגינה,כלי נגינה,[[...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...], [...]]
Done------------------------------------


C:\Users\PETER\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'techtop.co.il'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Scraping product data for אולפן DJ וכלי נגינה,כרטיסי קול,[[...], [...], [...], [...], [...], [...], [...], [...], [...], [...]]
Done------------------------------------


C:\Users\PETER\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'techtop.co.il'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Scraping product data for אולפן DJ וכלי נגינה,כרטיסי קול,[[...], [...], [...], [...], [...]]
Done------------------------------------
